# Notebook para treinar redes do DLC

A versãod o DLC que estamos usando aqui esta demorando muito pra instalar, ta tendo um bug que eu nao consigo tirar e ngm reclamou ainda no forum. Demora mas funciona. Como estamos usando uma máquina virtual, sempre que a gente inicializar ela, temos que instalar o que vamos usar. 

**Para rodar as células é só clicar no play no canto esquerdo de cada uma.**

**Sempre lembrar de montar o drive**
1.   << Aqui na esquerda tem uma pastinha
2.   vai abrir a aba arquivos
3.   esta aba tem 3 arquivos o terceiro é para montar o drive
4.   ao clicar nela ela ira montar automaticamente e vai aparecer a pasta do seu drive para pegar os arquivos do projeto.


Algumas coisas do python: 


*   Vermelho =  é uma string, no nosso caso aqui todos os caminhos. Tem que usar entre aspas
*   '#' = comentário, o que estiver depois da '#' não é lido pelo copilador
*   Verde = int/float, um número inteiro no nosso caso
*   Azul = variáveis reservadas do python que já possuem ação específica
*   Rosa = determina as condições do python



# Instalação

In [ ]:
#@title Install DLC
!pip install deeplabcut

     |████████████████████████████████| 541 kB 5.2 MB/s 
     |████████████████████████████████| 177 kB 50.0 MB/s 
     |████████████████████████████████| 948 kB 40.4 MB/s 
     |████████████████████████████████| 296 kB 46.9 MB/s 
     |████████████████████████████████| 15.7 MB 191 kB/s 
     |████████████████████████████████| 352 kB 44.4 MB/s 
     |████████████████████████████████| 109 kB 46.8 MB/s 
     |████████████████████████████████| 37.1 MB 50 kB/s 
     |████████████████████████████████| 29.2 MB 71 kB/s 
     |████████████████████████████████| 9.5 MB 38.8 MB/s 
     |████████████████████████████████| 546 kB 54.1 MB/s 
     |████████████████████████████████| 454.4 MB 10 kB/s 
     |████████████████████████████████| 462 kB 49.7 MB/s 
     |████████████████████████████████| 1.2 MB 37.9 MB/s 
     |████████████████████████████████| 4.0 MB 33.3 MB/s 
     |████████████████████████████████| 454.3 MB 18 kB/s 
     |████████████████████████████████| 394.5 MB 405 bytes/s 
     |███████

In [ ]:
#@title Imports
# %tensorflow_version 1.x
import os
os.environ["DLClight"]="True"

import deeplabcut
import numpy as np
import pandas as pd
deeplabcut.__version__

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


'2.2.0.1'

In [ ]:
#@title Somente para saber qual placa de vídeos estamos usando. Tesla K80 (menos poder) ou Tesla T4 (mais poder)

!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-0b0583a4-65f5-178b-3690-efa32bc237cf)


# Treinando a rede

Inserir aqui o caminho do arquivo de configuração 

In [ ]:
config_path = '/content/drive/MyDrive/SEU_ARQUIVO_DE_CONFIGURAÇÃO.yaml'

Essa célula cria o dataset de treino (95/5 ou de acordo com o passado no config.yaml).
Mesmo se o dataset de treino já tenha sido criado no computador, é preciso criar aqui de novo, pois existe uma diferença de aquitetura entre windows e linux (colab é baseado no linux). É um passo bem rápido que necessita uma confirmação, ao final da célular tera uma questão falando dos sistemas operacionais e pede para inserir 'y' ou 'n' para prosseguir, so colocar 'y' na caixa de digitação.

In [ ]:
deeplabcut.create_training_dataset(config_path)

Próxima célula é para treinar a rede em si. Essa função recebe muitos parâmetros, mas ultimamente esta sendo necessário utilizar somente três. Ela vai receber o arquivo de configuração como primeiro parâmetro; o segundo '**displayiters**' é para mostrar na tela o progresso do treino (Ex: *displayiters=2500*), a cada 2500 iterações ela vai sinalizar na saída da célula: *2500,5000,7500...*; o terceiro '**saveiters**' é um parâmetro de backup para salvar o progresso da rede, costumo deixar um valor baixo porque nao fico olhando o progresso da rede e o colab pode fechar a qualquer momento, independente da hora que ele fechar podemos retomar o treino de acordo com o último estado da rede. Esse progresso é salvo na pasta do DLC no drive e ele faz uma gestão de espaço, mas depois de treinado é só deletar os arquivos que não vão ser usados. Se em algum momento for treinar no próprio computador, não tem necessidade de salvar tão rápido, pode utilizar um valor de 25-30k (avaliar o parâmetro de acordo com a necessidade de uso do PC, geralmente treino a noite e espero o proximo checkpoint para parar a rede).

Talvez o único parâmetro que talvez seja necessário adicionar a função seja o **maxiters**, ele irá limitar a quantidade de iterações a serem feitas (Ex: *deeplabcut.train_network(config_path, displayiters=2500, saveiters=2500, maxiters=200000)*. O máxim de iterações que o DLC faz é 1 milhão se não me engano, geralmente eu treino entre 200-300k. Tenho achado interessante fazer 200k e refinar o labels (passos futuros) e re-treinar a rede.


**Do jeito que esta, esta configurado pra treinar a rede até o limite (1kk), mando treinar a rede e deixo até o limite do colab, umas 12h de GPU que da umas 170-200k de iterações ai ele para e da um aviso. Quando chegamos nesse limite temos que usar outra conta google (geralmente faço isso) para acessar GPU ou esperar 1 ou 2 dias para poder usar de novo.**

In [ ]:
deeplabcut.train_network(config_path, displayiters=2500, saveiters=2500)

# Analisando os videos

Para facilitar a análise, criei esse código que 'descobrir' todos os arquivos que estejam na pasta que terminem algum padrão, lembrar de alterar de acordo com o seu vídeo (Ex: ".AVI", ".mp4" ou outros). Todos os caminhos destes videos serão salvos em uma variável só para nao precisarmos ficar chamando a função várias vezes.


*   directory = pasta onde estão os videos, colocar nesta variável o caminho desta pasta
*   directoryResults= é o caminho onde você deseja que os arquivos sejam salvos, criar uma pasta e pegar estes caminhos

ps: Caso queira separar os video de outra forma, pode usar o começo do nome do arquivo como parâmetro para busca, caso haja uma sepração por animais e queira analisar so animais com um prefixo. Exemplo: analisar somente o rato 1 (R1D1,R1D2,R1D3...), pode substituir a função e trocar o *endswith()* para *startswith('R1')*

```
for filename in os.listdir(directory):
    if filename.startswith("R1"):
         listaPath.append(os.path.join(directory, filename))
```

In [ ]:
directory = '/content/drive/MyDrive/pasta_com_os_videos'
directoryResults = '/content/drive/MyDrive/pasta_onde_voce_deseja_salvar_os_resultados'

listaPath = []
for filename in os.listdir(directory):
    if filename.endswith(".AVI"): # MODIFICAR O FINAL DE ACORDO COM O FORMATO DO SEU VIDEO
         listaPath.append(os.path.join(directory, filename))

Próxima célula irá analisar os videos (decompor em frames e classificar os pontos), é um passo mais demorado. Ele irá processar tudo e salvar na pasta passada na variável '*directoryResults*', o único parâmetro que é necessário modificar aqui é o '*videotype*' para especificar qual o formato do vídeo. Os outros parâmetros já estão de acordo com as células anteriores, esta função também possui mais parâmetros mas raramente é necessário usar. Sempre importante ter o '*save_as_csv=True*' para termos essa informação das coordeandas em *csv*, se não ele salva somente formatos com uma codificação mais difícil de usar. A saida da função vai ser o *pickle*, um *h5* e o nosso *csv*

In [ ]:
dados_tracking = deeplabcut.analyze_videos(config_path,listaPath, videotype='.AVI', destfolder= directoryResults, save_as_csv=True)

Depois de realizar o tracking no passo anterior, podemos criar os vídeos com as marcações. A função já esta configurada para criar os videos de todos os videos passados para o tracking no passo anterior e salvar na mesma pasta de resultados. De um ponto de vista técnico não é necessário criar todos os videos, ai a função pode receber so alguns para ter um retorno visual de como ficou a qualidade. Pra isso é bom usar um video que a rede conhece (os frames foram extraídos dele para dar label) que os resultados serão melhores e um vídeo que a rede não conhece, para realmente testarmos a qualidade do vídeo.

De maneira geral, este passo é bem rápido. Costumo até fazer ele no meu computador por uma questão que ainda não consegui resolver aqui. Existe uma função de filtro do tracking, mas ainda não consegui utilizar ela por linha de código, não da erro mas tbm não filtra. Como esse filtro melhora o tracking, eu costumo fazer pelo meu notebook e utilizar minha GPU pra isso, pela interface tem como somente marcar a função de filtro e funciona bem.

In [ ]:
deeplabcut.create_labeled_video(config_path, listaPath, dados_tracking, destfolder= directoryResults)